# 测试 [pytest](https://docs.pytest.org/en/latest/) - part 2

In [1]:
# 请确保安装了 pytest 和 ipytest
# 在Jupyter笔记本中运行Pytest需要ipytest
import sys
!{sys.executable} -m pip install pytest
!{sys.executable} -m pip install ipytest

import ipytest
import pytest
__file__ = '测试2.ipynb'

## [`@pytest.fixture`](https://docs.pytest.org/en/latest/fixture.html#pytest-fixtures-explicit-modular-scalable)
考虑一下，我们有一个我们要测试的Person类的实现。

In [2]:

class Person:
    def __init__(self, first_name, last_name, age):
        self.first_name = first_name
        self.last_name = last_name
        self.age = age
    
    @property
    def full_name(self):
        return '{} {}'.format(self.first_name, self.last_name)
    
    @property
    def as_dict(self):
        return {'name': self.full_name, 'age': self.age}
        
    def increase_age(self, years):
        if years < 0:
            raise ValueError('不能让人年轻 :(')
        self.age += years

你可以使用pytest固定装置轻松创建可重用的测试代码。
如果你在 [_conftest.py_](https://docs.pytest.org/en/latest/fixture.html#conftest-py-sharing-fixture-functions)中引入固定装置，则这些固定装置可用于所有测试用例。
通常，conftest.py的位置位于测试目录的根目录。

In [3]:
# This would be in either conftest.py or test_person.py
@pytest.fixture()
def default_person():
    person = Person(first_name='John', last_name='Doe', age=82)
    return person

然后，您可以在实际测试用例中使用`default_person`固定装置。

In [4]:
%%run_pytest[clean]

# These would be in test_person.py
def test_full_name(default_person): # Note: we use fixture as an argument of the test case
    result = default_person.full_name
    assert result == 'John Doe'
    
    
def test_as_dict(default_person):
    expected = {'name': 'John Doe', 'age': 82}
    result = default_person.as_dict
    assert result == expected
    
    
def test_increase_age(default_person):
    default_person.increase_age(1)
    assert default_person.age == 83
    
    default_person.increase_age(10)
    assert default_person.age == 93
    
    
def test_increase_age_with_negative_number(default_person):
    with pytest.raises(ValueError):
        default_person.increase_age(-1)

UsageError: Cell magic `%%run_pytest[clean]` not found.


通过使用夹具，我们可以对所有四个测试用例使用相同的`default_person`！
在`test_increase_age_with_negative_number`中，我们使用[`pytest.raises`](https://docs.pytest.org/en/latest/assert.html#assertions-about-expected-exceptions) 来验证是否引发了异常。

## [`@pytest.mark.parametrize`](https://docs.pytest.org/en/latest/parametrize.html#pytest-mark-parametrize-parametrizing-test-functions)
有时你想使用多个不同的输入来测试相同的功能。 `pytest.mark.parametrize`是用于定义具有预期输出的多个不同输入的解决方案。让我们考虑以下`replace_names`函数的实现。

In [5]:
# This would be e.g. in string_manipulate.py
def replace_names(original_str, new_name):
    """Replaces names (uppercase words) of original_str by new_name"""
    words = original_str.split()
    manipulated_words = [new_name if w.istitle() else w for w in words]
    return ' '.join(manipulated_words)

我们可以使用`pytest.mark.parametrize`通过多个输入来测试`replace_names`函数

In [ ]:
%%run_pytest[clean]

# 这将在你的测试模块中
@pytest.mark.parametrize("original,new_name,expected", [
        ('this is Lisa', 'John Doe', 'this is John Doe'),
        ('how about Frank and Amy', 'John', 'how about John and John'),
        ('no names here', 'John Doe', 'no names here'),
    ])
def test_replace_names(original, new_name, expected):
    result = replace_names(original, new_name)
    assert result == expected